In [3]:
import os, sys
#current_dir = os.getcwd()
HOME_DIR = '/home/denys/projects/ml-models/kaggle/dog-breed-identification/'
DATA_HOME_DIR = HOME_DIR + 'data/'
#DATA_HOME_DIR = HOME_DIR + '/data/sample/'

In [4]:
from glob import glob
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold

from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.models import Sequential
from keras.models import Model
from keras.optimizers import Adam

from keras.regularizers import l2

Using TensorFlow backend.


In [5]:
%matplotlib inline

from keras.preprocessing import image
from sklearn.model_selection import KFold

import utils; reload(utils)
from utils import plot_learning_curve
from utils import save_array, load_array, onehot
from utils import get_batches, get_data,  get_breed

In [6]:
labels_df = pd.read_csv("data/labels.csv")

In [5]:
train_data = get_data(DATA_HOME_DIR+'train')

TypeError: get_data() got an unexpected keyword argument 'target_size'

In [10]:
save_array(DATA_HOME_DIR + 'train_244_244.data', train_data)

In [7]:
train_data = load_array(DATA_HOME_DIR+'train_244_244.data')

In [9]:
target_size = (244,244)
batches = get_batches(DATA_HOME_DIR + 'train', shuffle=False, batch_size=1, target_size=target_size)

Found 9225 images belonging to 120 classes.


In [10]:
train_classes = batches.classes
train_labels = onehot(train_classes)
train_filenames = batches.filenames
train_class_indices = batches.class_indices

In [11]:
model_path = DATA_HOME_DIR + "models/"
logs_path = HOME_DIR + "logs/"

In [12]:
kf = KFold(n_splits=3, shuffle=False, random_state=1) #shuffle=False to avoid data leakega during assamble
kf.get_n_splits(train_data)

3

In [13]:
from keras.callbacks import CSVLogger
model_name = "vgg16"
date_of_start = "0203_1625"
csv_logger = CSVLogger(logs_path + model_name + date_of_start + '.log', append=True)

In [14]:
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.mobilenet import MobileNet

# = {"vgg16": VGG16(weights='imagenet', include_top=False),
         # "resnet50" : ResNet50(weights='imagenet', include_top=False),
          #"InceptionV3": InceptionV3(weights='imagenet', include_top=False),
          #"InceptionResNetV2": InceptionResNetV2(weights='imagenet', include_top=False)}
 #        }

In [17]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model

base_model = VGG16(weights='imagenet', include_top=False)
print model_name
x = BatchNormalization()(base_model.output)
x = GlobalAveragePooling2D()(x)
x = Dense(200, activation='relu')(x)
x = BatchNormalization()(x)
predictions = Dense(120, activation='softmax')(x)
    
model = Model(inputs=base_model.input, outputs=predictions)
    
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', metrics=["accuracy"], loss='categorical_crossentropy')
    
model.load_weights(model_path + model_name + '_0203_1402.h5')
print model.summary()    

vgg16
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
____

In [ ]:
batch_size = 12
for train_index, valid_index in kf.split(train_data):
#    model.optimiser.lr = 0.0000001

    train_split, valid_split = train_data[train_index], train_data[valid_index]
    train_labels_split, valid_labels_split = train_labels[train_index], train_labels[valid_index]

        #rotation_range=15, width_shift_range=0.1, 
                               #height_shift_range=0.1, zoom_range=0.2, horizontal_flip=True
    batches = image.ImageDataGenerator(rotation_range=15, width_shift_range=0.1, 
                                       height_shift_range=0.1, zoom_range=0.2, horizontal_flip=True).flow(train_split, train_labels_split, batch_size=batch_size, shuffle=True)
    val_batches = image.ImageDataGenerator().flow(valid_split, valid_labels_split, batch_size=batch_size, shuffle=False)
    
    
    model.fit_generator(batches, steps_per_epoch=batches.n / batch_size, epochs=10, validation_data=val_batches, 
                        validation_steps=val_batches.n, verbose=1, callbacks=[csv_logger])
            
    model.save_weights(model_path + model_name + date_of_start + '.h5')
   # print model.test_on_batch(valid_data, valid_labels)

Epoch 1/10
512/512 [==============================] - 534s 1s/step - loss: 1.5589 - acc: 0.5807 - val_loss: 0.2546 - val_acc: 0.9208
Epoch 2/10
512/512 [==============================] - 532s 1s/step - loss: 1.3038 - acc: 0.6323 - val_loss: 0.3773 - val_acc: 0.8783
Epoch 3/10
512/512 [==============================] - 532s 1s/step - loss: 1.1575 - acc: 0.6585 - val_loss: 0.4802 - val_acc: 0.8545
Epoch 4/10
512/512 [==============================] - 532s 1s/step - loss: 1.0944 - acc: 0.6826 - val_loss: 0.5704 - val_acc: 0.8225
Epoch 5/10
512/512 [==============================] - 532s 1s/step - loss: 1.0042 - acc: 0.6991 - val_loss: 0.6854 - val_acc: 0.7844
Epoch 6/10
512/512 [==============================] - 532s 1s/step - loss: 0.9856 - acc: 0.7074 - val_loss: 0.7455 - val_acc: 0.7774
Epoch 7/10
511/512 [============================>.] - ETA: 0s - loss: 0.9345 - acc: 0.7244

In [26]:
plot_learning_curve(log_path + "vgg160203_1402.log")

NameError: name 'log_path' is not defined

In [18]:
def prepare_predictions(pred_probs):
    prepared_result = []
    max_probs = get_max_prob_per_item(pred_probs)
    for row, max_prob in zip(pred_probs, max_probs):
        prepared_result.append(np.vectorize(lambda prob: prob if prob == max_prob else 0.0)(row))
    return prepared_result

In [19]:
gen = image.ImageDataGenerator()
test_batches = gen.flow_from_directory(DATA_HOME_DIR + "test", target_size=target_size, batch_size=1, shuffle=False)

Found 10357 images belonging to 2 classes.


In [20]:
test_batches.reset()
filenames = test_batches.filenames
test_probs = model.predict_generator(test_batches, len(filenames))

In [242]:
len(test_probs)
print np.argmax(test_probs[54])
print test_batches.filenames[54]

90
unknown/012ca7efe684c5cdfb83f35e8fbafe1b.jpg


In [21]:
def get_ids(filenames):
    return np.vectorize(lambda filename: filename[-36:-4])(filenames)

In [22]:
test_ids = get_ids(test_batches.filenames)
len(test_ids)

10357

In [23]:
header = sorted(train_class_indices.keys())
header.insert(0, "id")
header = ','.join(header)

In [24]:
result = np.c_[test_ids, np.clip(test_probs, 0.05, 0.9)]

In [25]:
np.savetxt('vgg_02031402_clip.csv', result, delimiter=',', header=header, comments="", fmt='%s')

In [22]:
import keras

input1 = keras.layers.Input(shape=(16,))
x1 = keras.layers.Dense(8, activation='relu')(input1)
input2 = keras.layers.Input(shape=(32,))
x2 = keras.layers.Dense(8, activation='relu')(input2)
added = keras.layers.Add()([x1, x2])  # equivalent to added = keras.layers.add([x1, x2])

out = keras.layers.Dense(4)(added)
model = keras.models.Model(inputs=[input1, input2], outputs=out)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 16)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 8)            136         input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 8)            264         input_2[0][0]                    
__________________________________________________________________________________________________
add_1 (Add

In [ ]:
conv_pool_cnn_model = conv_pool_cnn(model_input)
all_cnn_model = all_cnn(model_input)
nin_cnn_model = nin_cnn(model_input)

conv_pool_cnn_model.load_weights('weights/conv_pool_cnn.29-0.10.hdf5')
all_cnn_model.load_weights('weights/all_cnn.30-0.08.hdf5')
nin_cnn_model.load_weights('weights/nin_cnn.30-0.93.hdf5')

models = [conv_pool_cnn_model, all_cnn_model, nin_cnn_model]

def ensemble(models, model_input):
    
    outputs = [model.outputs[0] for model in models]
    y = Average()(outputs)
    
    model = Model(model_input, y, name='ensemble')
    
    return model
ensemble_model = ensemble(models, model_input)